# Fruit Classification - 6 Classes (Berries & Fruits)

Training a MobileNetV2 model to classify: Blackberrie, Orange, Peach, Pear Williams, Raspberry, Strawberry

## Setup & Imports

In [1]:
!pip install tensorflow numpy matplotlib

In [ ]:
!pip uninstall protobuf


Found existing installation: protobuf 6.33.2
Uninstalling protobuf-6.33.2:
  Would remove:
    /home/youssefmoustaid/venv/lib/python3.12/site-packages/google/_upb/_message.abi3.so
    /home/youssefmoustaid/venv/lib/python3.12/site-packages/google/protobuf/*
    /home/youssefmoustaid/venv/lib/python3.12/site-packages/protobuf-6.33.2.dist-info/*
Proceed (Y/n)? 

In [ ]:
!pip install  protobuf

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")

## Data Configuration

In [ ]:
# Hyperparameters
BATCH_SIZE = 128
IMAGE_SIZE = (96, 96)  # MobileNetV2 optimal size
EPOCHS = 20  # More epochs for better learning

# Dataset paths
DATA_DIR = './fruits-360'
TRAIN_DIR = os.path.join(DATA_DIR, 'Training')
TEST_DIR = os.path.join(DATA_DIR, 'Test')

print(f"Training data: {TRAIN_DIR}")
print(f"Test data: {TEST_DIR}")

## Data Loading & Augmentation

In [ ]:


# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of training data for validation
)

# Only rescaling for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Load validation data from training set
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

NUM_CLASSES = train_generator.num_classes
print(f"\nTotal classes: {NUM_CLASSES}")
print(f"Classes: {list(train_generator.class_indices.keys())}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")

## Model Architecture

In [ ]:
# Load pre-trained MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Freeze base model
base_model.trainable = False

print(f"Base model loaded: {base_model.name}")
print(f"Base model layers: {len(base_model.layers)}")

In [ ]:
# Build model with custom head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])

model.summary()

In [ ]:
# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled and ready for training!")

## Training

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    verbose=1
)

print("\nTraining completed!")

## Evaluation & Visualization

In [ ]:
# Plot training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy', linewidth=2)
plt.plot(epochs_range, val_acc, label='Validation Accuracy', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss', linewidth=2)
plt.plot(epochs_range, val_loss, label='Validation Loss', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Training Accuracy: {acc[-1]:.4f}")
print(f"Final Validation Accuracy: {val_acc[-1]:.4f}")

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

## Save Models

In [ ]:
# Save Keras model
model.save('fruit_classifier_6class.h5')
print("Keras model saved: fruit_classifier_6class.h5")

## Convert to TensorFlow Lite

In [ ]:
# Convert to TFLite with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save TFLite model
tflite_path = 'fruit_classifier_6class.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved: {tflite_path}")
print(f"Model size: {len(tflite_model) / 1024 / 1024:.2f} MB")

## Save Label Map

In [ ]:
# Save class labels in correct order
labels = sorted(train_generator.class_indices.items(), key=lambda x: x[1])
class_names = [name for name, index in labels]

labels_path = 'fruit_classifier_6class_labels.txt'
with open(labels_path, 'w') as f:
    f.write('\n'.join(class_names))

print(f"Labels saved: {labels_path}")
print(f"\nClass names ({len(class_names)}):")
for i, name in enumerate(class_names):
    print(f"  {i}: {name}")